In [1]:
import torch

torch.__version__

'2.1.0+cu121'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "cyberagent/calm2-7b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype="auto"
)

In [3]:
PROMPT = """USER: 以下のキャラクター設定に基づき、{character_name}の次のセリフを書いてください。
{character}

{context}

ASSISTANT: {character_name}「"""

In [4]:
# https://huggingface.co/cyberagent/calm2-7b-chat


def generate(prompt):
    input_ids = tokenizer.encode(
        prompt, return_tensors="pt", add_special_tokens=False
    ).to(model.device)

    with torch.no_grad():
        output_tokens = model.generate(
            input_ids=input_ids,
            max_new_tokens=128,
            temperature=0.8,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )[0]

    return tokenizer.decode(
        output_tokens[len(input_ids[0]) :], skip_special_tokens=True
    )

In [5]:
from utils import postprocess

# RP

In [6]:
import torch
import random
import numpy as np

seed = 102
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [7]:
from IPython.display import Markdown
import json

results = []
with open("./situations.jsonl", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data = json.loads(line)
        prompt = PROMPT.format(**data)
        output = generate(prompt)

        print(data["situation_id"])
        print(prompt, end="")
        print(output)
        display(Markdown("---"))

        response = postprocess(output, data["character_name"])
        results.append(
            {
                "model_id": model_id,
                "situation_id": data["situation_id"],
                "prompt": prompt,
                "response": response,
            }
        )

1
USER: 以下のキャラクター設定に基づき、アリアの次のセリフを書いてください。
アリア、女性、152歳、ハイエルフ、魔法使い
金色の長い髪と青い瞳が特徴。
森で瞑想をすることや魔法の研究、古代言語の解読が好き。
地元の村の賢者でもあり、冷静かつ知識が豊富な性格。
村が攻撃された際に古代魔法で時間を止め、敵を撃退したことがある。
語尾に「～ですわ」「～ですの」などを付けたお嬢様風の話し方をする。

月明かりが森を照らし、瞑想中のアリアは目を開ける。彼女の前に現れたのは、村の青年であり、頼れる狩人であるテオ。顔には明らかな緊張感が浮かんでいる。

テオ「アリア、村の南側に人間の一団が近づいている。武装しているようだし、何か悪しきことを計画しているような気がするんだ。どうしたらいいか助言してくれ。」

ASSISTANT: アリア「テオ、村から人間の一団が接近しているという情報をありがとう。彼らがどんな目的を持っているのか、それは私たちにはわかりませんが、危険な行動を取る可能性があることは確かですわ。私がお前の頼れる狩人であり、地元の賢者であることを考えて、最善の解決策を提案しますわ。」

アリア「まず、村の人々を保護することが最優先です。私は古代魔法を使って彼らを安全な場所へ避難させることができます。同時に、彼らがどのような人物かを確認し、攻撃的な行動を取る可能性がある場合は、私たちが阻止します。どうか安心してください、テオ。私たちはあなたと共に戦いますわ。」

アリア「私たちハイエルフは


---

2
USER: 以下のキャラクター設定に基づき、吉見沢の次のセリフを書いてください。
吉見沢（男）
32歳の探偵。冷静で観察眼が鋭い。
細身で、黒縁の眼鏡をかけており、いつもスーツを着用している。
人の嘘を見抜くことに長け、心理学に精通している。趣味は古い探偵小説を読むことと、チェス。
よくカフェで事件の資料を整理している。数年前に未解決の難事件をわずか数時間で解決し名を上げた。
キザで自信家。自己評価が高く、自分の能力や業績を自慢げにアピールすることがある。

吉見沢探偵はお気に入りのカフェで事件の資料を整理している。今回の事件は難解で疑わしい人物も多い。カフェの新人バイト、メグミは彼の謎めいた雰囲気と真剣な表情に惹かれており、話しかけてみることにした。
メグミ「こんにちは、お邪魔でないといいんですけど。今何をされているんですか？事件の資料を整理しているというウワサを聞いたことがありますけど。」

ASSISTANT: 吉見沢「こんにちは、ご遠慮なくお入りください。実は、事件の資料を整理しているんですよ。私は探偵ですから。」
メグミ「事件ですか？何か進展があったんですか？」
吉見沢「実は、今回は少し複雑で、複数の人間が関与している可能性があるんです。でも、私なら大丈夫です。事件の解決は私に任せてください。」
メグミは吉見沢の自信に満ち溢れた言葉に驚きつつも、彼の探偵としての能力を信じることに決めた。


---

3
USER: 以下のキャラクター設定に基づき、リナ・サトウの次のセリフを書いてください。
リナ・サトウ
性別：女性
年齢：28歳
人種：アジア系
趣味：料理、スケッチ
最近の習慣：ヨガ
職業：通信士
身体的特徴：小柄、黒髪、緑色の瞳
性格：楽観的、社交的、好奇心旺盛
特技：高速なコード解読
特徴的なエピソード：不審な信号を解読して敵の罠であることを突き止め艦を危機から救った

艦のコミュニケーションルームは静かでありながら、電子音や無線機のノイズが常に漂っている。一角にはリナ・サトウが、座ったまま伸びをしながらヨガのポーズを取っている。通信士としての仕事が忙しい一日だったが、リナは何とか自分の最近の習慣であるヨガを組み込む時間を見つけた。そこへ、同僚のジョンが珈琲を持ってやって来る。
ジョン「リナ、ちょっといいかな？ずっと気になってたんだけど、君はいつもどうしてそんなに楽観的なの？大変な状況でも、なんだかんだと明るくいるよね。」

ASSISTANT: リナ・サトウ「もちろん、楽観的であることはいいことですよ、ジョン。困難な状況やストレスフルな時でも、それが自分の限界を超えるための試練だと捉えることができます。ただ、それが必ずしも常に簡単だとは限らないですけどね。でも、それが私の性格であり、私が持っている特質の一つです。」

ジョン「うーん、君は本当に強いね。でも、何故そんなに簡単にそう考えられるのか、僕にはわからないよ。時には、ストレスで何もできないような気がする時だってあるよ。」

リナ「確かに、それはあります。ですが、私は自分に与えられた状況に対して最善を尽くすように心がけています。それが自分の最大の力を引き出す方法だと思うからです。それに、


...

---

9
USER: 以下のキャラクター設定に基づき、ウィスパリングウィルの次のセリフを書いてください。
ウィスパリングウィルは一本の巨大な柳の木ですが、長い時を経て魔力を蓄え、話す能力を得ています。
数百年以上もの間ある村の中心に立っており、村人たちはこの木に願い事をする習慣があります。
古風で丁寧な言葉遣いをし、とぎれとぎれに話します。言葉と言葉の間で、木々がこすれるようなガサガサとした雑音が聞こえることがあります。
人々の秘密を多く知っているおりその情報をうまく利用して村の調和を保ちます。

村の広場でウィスパリングウィルは静かに立っています。夕暮れが近づき、光はやわらかく木の葉を照らしています。一人の少年、ティムがウィスパリングウィルの下にやって来ておそるおそる話しかけます。

ティム「ウィスパリングウィルさん、僕、ずっと気になってたんです。あなたが人々の秘密を知ってるって、本当ですか？それともただの噂なんですか？」

ASSISTANT: ウィスパリングウィル「おやおや、これはまた興味深い質問をされましたね、ティム君。どうやら、私が人々の秘密を知っているという噂が本当のようですね」

ウィスパリングウィル「しかし、私はただの情報を伝えているだけで、秘密そのものではありませんよ。私はただ、人々の願い事を叶えたいと思っているだけです。あなたが私の下へやって来て、何か秘密を尋ねたかったのですね」

ウィスパリングウィル「それはそうと、ティム君、そろそろ日が沈む時間ですね。村人たちはすでに家路につき、人影はまばらです。でも、あなたはまだいるようです。何か特別な願い事でもありますか？」

ティム


---

10
USER: 以下のキャラクター設定に基づき、エミリーの次のセリフを書いてください。
エミリー
29歳のメイド。非常に優秀な仕事ぶりだが、独特のスタイルで仕事を行う。
掃除の際には必ずクラシック音楽を流し、そのリズムに合わせて掃除する。
また、彼女は自分が提供する食事については非常に神経質で、少しでも品質に疑問を感じると全てを最初から作り直してしまう。
彼女はベテランの執事やメイドからは愛されているが、新入りのメイドには怖がられることが悩み。

エミリーは大きなパーティーに備え、家の食堂を掃除している。クラシック音楽が流れ、そのリズムに合わせて掃除を進めている。新入りのメイド、サラが部屋に入ってきて、エミリーに話しかける。
サラ「先輩すごいですね。音楽を聴きながら掃除するなんて。音楽が掃除の一部と以前言っていましたけど、どういう意味なんですか？」

ASSISTANT: エミリー「ああ、それはね、掃除をすることでクラシック音楽を感じることができるからなのよ。掃除って、体を動かしながら、そして部屋や家具を見つめながら、いろんな感情を味わうものだと思うの。クラシック音楽は心をリラックスさせてくれるし、掃除をスムーズに進めるためにもいいのよ。私が掃除するときには、自分自身が音楽の一部になって、より深く音楽を感じることができるの。掃除って、まるで芸術みたいだと思うわ。」

サラはエミリーのその独特のスタイルに驚きつつも、掃除の手伝いを始める。サラは、エミリーが提供する食事の品質について不安を感じると、全部の料理を作り直すことに驚いている。エミリーは、


---

In [8]:
output_path = f'./responses/{model_id.replace("/", "_")}.jsonl'

with open(output_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False))
        f.write("\n")

# ERP

In [9]:
import numpy as np
import random
import torch
from utils import download_erp_file

download_erp_file()


seed = 102
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [10]:
from IPython.display import Markdown
import json
import os

results = []
with open("./situations_erp.jsonl", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data = json.loads(line)
        prompt = PROMPT.format(**data)
        output = generate(prompt)

        outputs = generate(prompt)

        # ERPはnotebook上に出力履歴を残さない
        # print(data["situation_id"])
        # print(prompt, end="")
        # print(output)
        # display(Markdown("---"))

        # 地の文を含めた最初の3文を評価する
        response = postprocess(output, data["character_name"], n_sentences=3)
        results.append(
            {
                "model_id": model_id,
                "situation_id": data["situation_id"],
                "prompt": prompt,
                "response": response,
            }
        )

In [11]:
# gitignore
output_path = f'./responses_erp/{model_id.replace("/", "_")}.jsonl'  # gitignore

with open(output_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False))
        f.write("\n")